In [1]:
# bs4 nltk gensim
import os
import re
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn 
from nltk import SnowballStemmer
from nltk.stem import WordNetLemmatizer
from bs4 import BeautifulSoup
#nltk.download()

In [24]:
#去掉HTML标签的数据
#example = BeautifulSoup(df['review'][1000], 'html.parser').get_text()
class EnPreprocess:  
    def FileRead(self, filePath):#读取内容 
        with open(filePath,encoding='UTF-8',errors='ignore') as f:  
        #f = open(filePath)  
            raw = f.read()
            f.close()

        return raw  
    
    def WriteResult(self, result, resultPath): 
        Path = str(resultPath).split('/')[0] + '/' + 'transform' + '/'
        print("writepath::::", Path+'PMP books.txt')
        #Path = str(resultPath).replace(str(resultPath).split('/')[-1],'') + 'transform' + '/'
        self.mkdir(Path)
        #self.mkdir(str(resultPath).replace(str(resultPath).split('/')[-1],''))
        f = open(Path+'PMP books.txt','a+',encoding='utf8') #将结果保存到另一个文档中  
        for i in result:
            if i != '\n':
                f.write(str(i)+' ')
            else:
                f.write(str(i))
        #f.write(str(result)) 
        f.close()
        
    def mkdir(self, path):  
        # 去除首位空格  
        path = path.strip()  
        # 去除尾部 \ 符号  
        path = path.rstrip("\\")  
        # 判断路径是否存在  
        # 存在    True  
        # 不存在  False  
        isExists = os.path.exists(path)
        
        # 判断结果  
        if not isExists:  
            # 如果不存在则创建目录  
            print(path+'  create successfully!')
            print('\n')
            # 创建目录操作函数  
            os.makedirs(path)  
            return True  
        else:  
            # 如果目录存在则不创建，并提示目录已存在  
            #print(path+'  The directory exists!!')
            print('\n')
            return False 
        
    def getHtml(url):
        page = urllib.request.urlopen(url)  
        html = page.read().decode("utf-8")
        soup=bs4.BeautifulSoup(html,'html.parser')
        soup.prettify()
        tds = soup.find_all('td')

        td_content=[]
        for i in tds:
            #print(i.text) #这里取标签span的内容
            td_content.append(i.text)
        #print(td_content) 
        return html

        
    def SenToken(self, raw):#分割成句子  
        sent_tokenizer = nltk.data.load('CopusPreprocess/tokenizers/punkt/english.pickle')  
        sents = sent_tokenizer.tokenize(raw)
        return sents
    
    def POSTagger(self, sent):  
        taggedLine = [nltk.pos_tag(sent) for sent in sents]  
        return taggedLine
    
    def WordTokener(self, sent):#将单句字符串分割成词  
        result=''  
        wordsInStr = nltk.word_tokenize(sent)
        return wordsInStr
    
    def WordCheck(self, words):#拼写检查  
        d = enchant.Dict("en_US")  
        checkedWords = ()  
        for word in words:  
            if notd.check(word):  
                d.suggest(word)  
                word = raw_input()  
            checkedWords = (checkedWords,'05')  
        return checkedWords
    
    def CleanLines(self, line):  #去除数字和ACSCII符号
        a = '[’!"#$%&\'•●®()*+,-./:;<=>?@[\\]^_`{|}~]+'    
        line = re.sub(a, " ", line)
        line1 = re.sub('\d'," ",line) #数字
        line2 = re.sub('\n'," ",line1) #去掉换行符
        cleanLine = re.sub('[^a-zA-Z]',' ',line2)
        
        return cleanLine  
    
    def CleanWords(self, wordsInStr):#小写化，去掉停用词  
        cleanWords = []  
          
        for words in wordsInStr:  
            cleanWords += [[w.lower() for w in words if w.lower() not in stopwords.words('english') and 3<=len(w)]]
         
            #cleanWords += [w.lower() for w in words if w.lower() not in stopwords.words('english')]
            #cleanWords += [w.lower() for w in words if len(w)>=3]
            
        return cleanWords  
    
    def StemWords(self, cleanWordsList):  #提取词干
        stemWords = []  
        #porter = nltk.PorterStemmer() 
        #stemWords = [porter.stem(t) for t in cleanWordsList]  
        
        #for words in cleanWordsList:  
            #stemWords += [[wn.morphy(w) for w in words]]  
            
        s = nltk.stem.SnowballStemmer('english')  
        stemWords = [s.stem(t) for t in cleanWordsList]  
        return stemWords
    
    def LemmatizeWords(self, cleanWordsList): #词性还原
        lemmatizeWords = []
        l = WordNetLemmatizer()
        lemmatizeWords = [l.lemmatize(t) for t in cleanWordsList]
        return lemmatizeWords
        
    def WordsToStr(self, stemWords):  #转换成字符串
        num = 0
        strLine = []
        for words in stemWords:
            for w in words:
                num += 1
                if num > 20:
                    strLine.append('\n')
                    num = 0
                else:
                    strLine.append(w)
            #strLine += [w for w in words]
        
        
        return strLine  
        
    def EnPreMain(self, dir):
        for root,dirs,files in os.walk(dir):
            for eachfiles in files:  
                dataPath = root + '/' + eachfiles
                print("CopusPath::::", dataPath)
                raw = self.FileRead(dataPath).strip()
                sents = self.SenToken(raw) #分句
    #             taggedLine=self.POSTagger(sents)#暂不启用词性标注
                cleanLines = [self.CleanLines(line) for line in sents] #清洗句子，去掉标点符号，数字
                #print(cleanLines)
                words = [self.WordTokener(cl) for cl in cleanLines]#分词
    #            checkedWords=self.WordCheck(words)#暂不启用拼写检查
                cleanWords = self.CleanWords(words) #转化为小写并且去掉停用词
                #stemWords = self.StemWords(cleanWords)#提取词干
                #lemmatizeWords = self.LemmatizeWords(cleanWords)#词性还原
                #strLine = self.WordsToStr(stemWords)#重新合成为句子
                strLine = self.WordsToStr(cleanWords)#重新合成为句子
                self.WriteResult(strLine,dataPath)  
  
        
    def StandardTokener(self, raw):  
        result=''   
        return result 
       

In [25]:
enPre=EnPreprocess()  
enPre.EnPreMain('CopusPreprocess/data')  


CopusPath:::: CopusPreprocess/data/PMP books.txt
writepath:::: CopusPreprocess/transform/PMP books.txt


CopusPath:::: CopusPreprocess/data/PMP Project Management Professional Study Guide1.txt
writepath:::: CopusPreprocess/transform/PMP books.txt


